In [1]:
'''
for ranges look up into "split_by_heights.ipynb"
'''

import re 
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
# noise
def add_noise(image_path, noise_factor):
    image = Image.open(image_path)
    image = np.array(image)
    noise = np.random.normal(scale=noise_factor, size=image.shape)
    noisy_image = np.clip(image + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_image)


path_folder = f'C:\Stuff\Dataset-creation\data_lego\lego-noise'

for folder in os.listdir(path_folder):
    std = int(re.findall(r'\d+', folder)[0])
    print(folder)
    print(std)
    path_image_folder = os.path.join(path_folder, folder, 'train')
    for image in os.listdir(path_image_folder):
        # if 'train' in image:
            path_image = os.path.join(path_image_folder, image)
            noisy_image = add_noise(path_image, noise_factor=std)
            noisy_image.save(path_image)

lego-noise-std-112
112
lego-noise-std-128
128
lego-noise-std-16
16
lego-noise-std-32
32
lego-noise-std-48
48
lego-noise-std-64
64
lego-noise-std-80
80
lego-noise-std-96
96


In [3]:
colors_dict = {'red': 0, 'green': 1, 'blue': 2}

def get_true_indices(mask):
    return np.transpose(np.nonzero(mask))

def add_noise(image_path, salt_prob, pepper_prob, color):
    image = Image.open(image_path)
    salt_mask = np.random.rand(image.size[1], image.size[0]) < salt_prob
    pepper_mask = np.random.rand(image.size[1], image.size[0]) < pepper_prob

    indices_salt = get_true_indices(salt_mask)
    indices_pepper = get_true_indices(pepper_mask)

    noisy_image = np.array(image)

    color_id = colors_dict[color]

    noisy_image[indices_salt[:, 0], indices_salt[:, 1], color_id] = 255.0
    noisy_image[indices_pepper[:, 0], indices_pepper[:, 1], color_id] = 0.0

    return Image.fromarray(noisy_image)

path_folder = 'C:\\Stuff\\Dataset-creation\\data_lego\\lego-pns-noise'

for color in ['green', 'blue', 'red']:
    for folder in os.listdir(path_folder):
        if color in folder:
            std = int(re.findall(r'\d+', folder)[0])/100
            salt_prob = pepper_prob = std
            path_image_folder = os.path.join(path_folder, folder, 'train')
            for image in os.listdir(path_image_folder):
                path_image = os.path.join(path_image_folder, image)
                noisy_image = add_noise(path_image, salt_prob, pepper_prob, color=color)
                noisy_image.save(path_image)
            print(f'done {folder} with std {std}')

done lego-pns-green-noise-percent-10 with std 0.1
done lego-pns-green-noise-percent-15 with std 0.15
done lego-pns-green-noise-percent-20 with std 0.2
done lego-pns-green-noise-percent-25 with std 0.25
done lego-pns-green-noise-percent-30 with std 0.3
done lego-pns-green-noise-percent-35 with std 0.35
done lego-pns-green-noise-percent-40 with std 0.4
done lego-pns-green-noise-percent-5 with std 0.05
done lego-pns-blue-noise-percent-10 with std 0.1
done lego-pns-blue-noise-percent-15 with std 0.15
done lego-pns-blue-noise-percent-20 with std 0.2
done lego-pns-blue-noise-percent-25 with std 0.25
done lego-pns-blue-noise-percent-30 with std 0.3
done lego-pns-blue-noise-percent-35 with std 0.35
done lego-pns-blue-noise-percent-40 with std 0.4
done lego-pns-blue-noise-percent-5 with std 0.05
done lego-pns-red-noise-percent-10 with std 0.1
done lego-pns-red-noise-percent-15 with std 0.15
done lego-pns-red-noise-percent-20 with std 0.2
done lego-pns-red-noise-percent-25 with std 0.25
done leg

In [2]:
import re
import os 
import cv2
import numpy as np

def random_saturate(image_path, scale):
    image = cv2.imread(image_path, -1)
    if image.shape[2] == 4:  # Check if the image has 4 channels (RGBA)
        rgb_image, alpha_channel = image[:, :, :3], image[:, :, 3]
    else:  # If the image is not RGBA, we treat it as RGB (no alpha channel)
        rgb_image = image
        alpha_channel = None

    # Convert RGB image to HSV
    hsv_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2HSV).astype("float32")

    # Adjust saturation
    saturation_factor = np.random.normal(loc=1.0, scale=scale)
    hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1] * saturation_factor, 0, 255)

    # Convert back to RGB
    saturated_rgb = cv2.cvtColor(hsv_image.astype("uint8"), cv2.COLOR_HSV2BGR)

    # Merge the processed RGB channels back with the Alpha channel, if it existed
    if alpha_channel is not None:
        saturated_rgba = cv2.merge((saturated_rgb[:, :, 0], saturated_rgb[:, :, 1], saturated_rgb[:, :, 2], alpha_channel))
    else:  # If there was no alpha channel, return the RGB image
        saturated_rgba = saturated_rgb
    return saturated_rgba

path_folder = 'C:\Stuff\Dataset-creation\data_lego\lego-saturation'

for folder in os.listdir(path_folder):
    pattern = re.compile(r'\d+\.\d+')
    matches = pattern.findall(folder)
    std = float(matches[0])

    path_image_folder = os.path.join(path_folder, folder, 'train')
    for image in os.listdir(path_image_folder):
        path_image = os.path.join(path_image_folder, image)
        noisy_image = random_saturate(path_image, scale=std)
        cv2.imwrite(path_image, noisy_image)
    print(f'done {folder} with std {std}')            

done lego-saturation-std-0.1 with std 0.1
done lego-saturation-std-0.2 with std 0.2
done lego-saturation-std-0.3 with std 0.3
done lego-saturation-std-0.4 with std 0.4
done lego-saturation-std-0.5 with std 0.5
done lego-saturation-std-0.6 with std 0.6
done lego-saturation-std-0.7 with std 0.7
done lego-saturation-std-0.8 with std 0.8
done lego-saturation-std-0.9 with std 0.9
done lego-saturation-std-1.0 with std 1.0


In [5]:
import numpy as np 
from PIL import Image
import re
import os 
import random
def adjust_gamma(image_path, gamma=1.5):
    image = Image.open(image_path)
    img_array = np.array(image)
    adjusted_img_array = 255 * (img_array / 255) ** gamma
    adjusted_img_array = np.clip(adjusted_img_array, 0, 255).astype(np.uint8)
    adjusted_image = Image.fromarray(adjusted_img_array)
    return adjusted_image

path_folder = 'C:\Stuff\Dataset-creation\data_lego\lego-gamma'

gamma_increase = 0.2
gamma_decrease = 5
for folder in os.listdir(path_folder):
    value = int(re.findall(r'\d+', folder)[0])/100
    path_image_folder = os.path.join(path_folder, folder, 'train')

    train_images = [i for i in os.listdir(path_image_folder) if True]
    selected_images = random.sample(train_images, int(value * len(train_images)))
    for img_name in selected_images:
        path_image = os.path.join(path_image_folder, img_name)
        noisy_image = None
        if random.choice([True, False]):
            print(path_image)
            noisy_image = adjust_gamma(path_image, gamma_increase)
        # For the second half, decrease gamma
        else:
            noisy_image = adjust_gamma(path_image, gamma_decrease)
        noisy_image.save(path_image)
    print(f'done {folder} ')

C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-10\train\r_13.png
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-10\train\r_8.png
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-10\train\r_30.png
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-10\train\r_5.png
done lego-gamma-10 
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-15\train\r_3.png
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-15\train\r_12.png
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-15\train\r_55.png
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-15\train\r_23.png
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-15\train\r_37.png
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-15\train\r_14.png
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-15\train\r_36.png
done lego-gamma-15 
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego-gamma-20\train\r_42.png
C:\Stuff\Dataset-creation\data_lego\lego-gamma\lego

In [6]:
import re
import os
import shutil
import json
path_folder = 'C:\Stuff\Dataset-creation\data_lego\lego-sparce'
path_original_tf = 'C:\Stuff\Dataset-creation\data_lego\lego\\transforms_train.json'

for folder in os.listdir(path_folder):
    every_num = int(re.findall(r'\d+', folder)[0])

    path_transforms = os.path.join(path_folder, folder, 'transforms_train.json') 
    shutil.copy2(path_original_tf, path_transforms)
    f = open(path_transforms)
    file = json.load(f)
    new_frames = []
    
    i = 0
    for frame in file['frames']:
        if (i+1) % every_num == 0:
            new_frames.append(frame)
        i += 1
        
    file['frames'] = new_frames
    with open(path_transforms, "w") as jsonFile:
        json.dump(file, jsonFile, indent=4)     

    print(f'done {folder}') 

done lego-every-2
done lego-every-3
done lego-every-4
done lego-every-5
done lego-every-6
done lego-every-7
done lego-every-8


In [1]:
# import re
# import os
# import shutil
# import json
# path_folder = 'C:\Stuff\Dataset-creation\data_lego\lego-sparce-offset'
# path_original_tf = 'C:\Stuff\Dataset-creation\data_lego\lego-original\\transforms.json'

# for folder in os.listdir(path_folder):
#     every_num = int(re.findall(r'\d+', folder)[0])
#     path_transforms = os.path.join(path_folder, folder, 'transforms.json')
#     f = open(path_transforms)
#     file = json.load(f)
#     new_frames = []
#     i = 0
#     for frame in file['frames']:
#         if 'train' in frame['file_path']:
#             if (i+1) % every_num == 0:
#                 new_frames.append(frame)
#             i += 1
#         elif 'eval' in frame['file_path']: 
#             new_frames.append(frame)
#         else:
#             raise KeyboardInterrupt
        
#     file['frames'] = new_frames
#     with open(path_transforms, "w") as jsonFile:
#         json.dump(file, jsonFile, indent=4)     

#     print(f'done {folder}') 

done stump-offset-every-2
done stump-offset-every-3
done stump-offset-every-4
done stump-offset-every-5
done stump-offset-every-6
done stump-offset-every-7
done stump-offset-every-8


In [4]:
# import re
# import os
# import shutil
# import json
# path_folder = 'C:\Stuff\Dataset-creation\data_lego\lego-sparce-offset-5'
# path_original_tf = 'C:\Stuff\Dataset-creation\data_lego\lego-original\\transforms.json'

# for folder in os.listdir(path_folder):
#     every_num = int(re.findall(r'\d+', folder)[-1])
#     path_transforms = os.path.join(path_folder, folder, 'transforms.json')
#     f = open(path_transforms)
#     file = json.load(f)
#     new_frames = []
#     i = 0
#     for frame in file['frames']:
#         if 'train' in frame['file_path']:
#             if (i+1) % every_num == 0:
#                 new_frames.append(frame)
#             i += 1
#         elif 'eval' in frame['file_path']: 
#             new_frames.append(frame)
#         else:
#             raise KeyboardInterrupt
        
#     file['frames'] = new_frames
#     with open(path_transforms, "w") as jsonFile:
#         json.dump(file, jsonFile, indent=4)     

#     print(f'done {folder}') 

done stump-offset-5-every-2
done stump-offset-5-every-3
done stump-offset-5-every-4
done stump-offset-5-every-5
done stump-offset-5-every-6
done stump-offset-5-every-7
done stump-offset-5-every-8
